In [1]:
import os
from pathlib import Path

from arcgis.gis import GIS
from dotenv import find_dotenv, load_dotenv  # for loading variable from .env file to keep them out of version control
from modeling import Country  # used as the starting point for working with 
import pandas as pd

load_dotenv(find_dotenv())  # recursively crawls up the directory tree until a .env file is found

# load the "autoreload" extension so as src code is changed, the changes are picked up in the dataframe
%load_ext autoreload
%autoreload 2

In [2]:
project_parent = Path('.').absolute().parent.parent
dir_data = project_parent/'data'
dir_int = dir_data/'interim'
dir_raw = dir_data/'raw'

# make sure data directories are present - helps for when project has just been cloned
for ddir in [dir_int, dir_raw]:
    if not ddir.exists():
        ddir.mkdir(parents=True)

# although not necessary, dropping this list of columns later makes it easier to view business lising results
biz_drop_cols = ['OBJECTID', 'CONAME','SALESVOL', 'HDBRCH', 'ULTNUM', 'PUBPRV', 'EMPNUM', 'FRNCOD', 'ISCODE', 'SQFTCODE', 'LOC_NAME', 'STATUS', 'SCORE', 'SOURCE', 'REC_TYPE']

# Setup

Country, Area-of-Interest,  and Enrichment Variables

In [3]:
# connection to Web GIS being used, so have to create it
# NOTE - using values from .env, thus keeping any credentials out of version control
gis = GIS(os.getenv('BA_QA_URL'), username=os.getenv('BA_QA_USERNAME'), password=os.getenv('BA_QA_PASSWORD'))

# explicitly using connection to GIS
usa = Country('USA', source=gis)

# ...although the same can also be done using local resources, ArcGIS Pro with Business Analyst and locally installed data.
# usa = Country('USA', source='local')

usa

<modeling.Country - USA (GIS at https://baqa.mapsqa.arcgis.com logged in as jmccune_baqa)>

In [4]:
%%time

# next, get a study area, an area of interest to work with - this constrains results to a reasonable domain
# NOTE - eventually, as these service evolve, this also will have the ability to span international borders, so essential to the workflow
aoi_df = usa.cbsas.get('seattle')

aoi_df

CPU times: user 49.7 ms, sys: 6.96 ms, total: 56.6 ms
Wall time: 1.22 s


,ID,NAME,SHAPE
0,42660,"Seattle-Tacoma-Bellevue, WA Metropolitan Stati...","{""rings"": [[[-122.83937000030072, 47.256580000..."


In [5]:
# next, since from ArcGIS Online, there are 18e3 variables available, and from ArcGIS Desktop with USA data, there are 8e3, we need to at least select some varilables to focus on
evars = usa.enrich_variables

print(evars.info())

# ...so we are starting with the current year "Key" variables
key_vars = evars[
    (evars.name.str.endswith('CY'))
    & (evars.data_collection.str.lower().str.contains('key'))
].reset_index(drop=True)

print(key_vars.info())
key_vars

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18399 entries, 0 to 41
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               18399 non-null  object
 1   alias              18399 non-null  object
 2   data_collection    18399 non-null  object
 3   enrich_name        18399 non-null  object
 4   enrich_field_name  18399 non-null  object
 5   description        18306 non-null  object
 6   vintage            18297 non-null  object
 7   units              18399 non-null  object
dtypes: object(8)
memory usage: 1.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               20 non-null     object
 1   alias              20 non-null     object
 2   data_collection    20 non-null     object
 3   enrich_name        20 non-null     obj

,name,alias,data_collection,enrich_name,enrich_field_name,description,vintage,units
0,TOTPOP_CY,2020 Total Population,KeyUSFacts,KeyUSFacts.TOTPOP_CY,KeyUSFacts_TOTPOP_CY,2020 Total Population (Esri),2020,count
1,GQPOP_CY,2020 Group Quarters Population,KeyUSFacts,KeyUSFacts.GQPOP_CY,KeyUSFacts_GQPOP_CY,2020 Group Quarters Population (Esri),2020,count
2,DIVINDX_CY,2020 Diversity Index,KeyUSFacts,KeyUSFacts.DIVINDX_CY,KeyUSFacts_DIVINDX_CY,2020 Diversity Index (Esri),2020,count
3,TOTHH_CY,2020 Total Households,KeyUSFacts,KeyUSFacts.TOTHH_CY,KeyUSFacts_TOTHH_CY,2020 Total Households (Esri),2020,count
4,AVGHHSZ_CY,2020 Average Household Size,KeyUSFacts,KeyUSFacts.AVGHHSZ_CY,KeyUSFacts_AVGHHSZ_CY,2020 Average Household Size (Esri),2020,count
5,MEDHINC_CY,2020 Median Household Income,KeyUSFacts,KeyUSFacts.MEDHINC_CY,KeyUSFacts_MEDHINC_CY,2020 Median Household Income (Esri),2020,currency
6,AVGHINC_CY,2020 Average Household Income,KeyUSFacts,KeyUSFacts.AVGHINC_CY,KeyUSFacts_AVGHINC_CY,2020 Average Household Income (Esri),2020,currency
7,PCI_CY,2020 Per Capita Income,KeyUSFacts,KeyUSFacts.PCI_CY,KeyUSFacts_PCI_CY,2020 Per Capita Income (Esri),2020,currency
8,TOTHU_CY,2020 Total Housing Units,KeyUSFacts,KeyUSFacts.TOTHU_CY,KeyUSFacts_TOTHU_CY,2020 Total Housing Units (Esri),2020,count
9,OWNER_CY,2020 Owner Occupied HUs,KeyUSFacts,KeyUSFacts.OWNER_CY,KeyUSFacts_OWNER_CY,2020 Owner Occupied Housing Units (Esri),2020,count


# Who and Where

Block groups and enrich variables

In [6]:
%%time

# since we have function chaining at our disposal, we can get all the demographic polygons available at the lowest level of geography (block groups) and enrich them
bg_df = aoi_df.mdl.level(0).get().mdl.enrich(key_vars)

print(bg_df.info())
bg_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2480 entries, 0 to 2479
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   ID          2480 non-null   object  
 1   NAME        2480 non-null   object  
 2   TOTPOP_CY   1440 non-null   float64 
 3   GQPOP_CY    1440 non-null   float64 
 4   DIVINDX_CY  1440 non-null   float64 
 5   TOTHH_CY    1440 non-null   float64 
 6   AVGHHSZ_CY  1440 non-null   float64 
 7   MEDHINC_CY  1440 non-null   float64 
 8   AVGHINC_CY  1440 non-null   float64 
 9   PCI_CY      1440 non-null   float64 
 10  TOTHU_CY    1440 non-null   float64 
 11  OWNER_CY    1440 non-null   float64 
 12  RENTER_CY   1440 non-null   float64 
 13  VACANT_CY   1440 non-null   float64 
 14  MEDVAL_CY   1440 non-null   float64 
 15  AVGVAL_CY   1440 non-null   float64 
 16  POPGRW10CY  1440 non-null   float64 
 17  HHGRW10CY   1440 non-null   float64 
 18  FAMGRW10CY  1440 non-null   float64 
 19  DPOP_C

,ID,NAME,TOTPOP_CY,GQPOP_CY,DIVINDX_CY,TOTHH_CY,AVGHHSZ_CY,MEDHINC_CY,AVGHINC_CY,PCI_CY,...,VACANT_CY,MEDVAL_CY,AVGVAL_CY,POPGRW10CY,HHGRW10CY,FAMGRW10CY,DPOP_CY,DPOPWRK_CY,DPOPRES_CY,SHAPE
0,530330012001,530330012.001,1342.0,0.0,62.5,571.0,2.35,95936.0,129349.0,55036.0,...,50.0,643634.0,647973.0,0.91,0.73,0.94,1112.0,457.0,655.0,"{""rings"": [[[-122.31243049957185, 47.704870499..."
1,530330012002,530330012.002,1392.0,110.0,53.1,571.0,2.25,107276.0,149116.0,61313.0,...,23.0,686765.0,684551.0,0.82,0.77,0.99,1160.0,567.0,593.0,"{""rings"": [[[-122.3178410004258, 47.7076489995..."
2,530330032001,530330032.001,1298.0,0.0,45.6,515.0,2.52,132945.0,187017.0,74202.0,...,23.0,851415.0,924422.0,0.70,0.45,0.75,1104.0,447.0,657.0,"{""rings"": [[[-122.38754700019909, 47.675934999..."
3,530330032002,530330032.002,1115.0,2.0,36.7,474.0,2.35,108057.0,173682.0,73838.0,...,34.0,900240.0,993990.0,0.10,-0.14,0.21,982.0,400.0,582.0,"{""rings"": [[[-122.38758899913357, 47.671643999..."
4,530330032003,530330032.003,2365.0,801.0,60.9,775.0,2.02,19815.0,43614.0,15269.0,...,27.0,496667.0,813983.0,1.72,1.17,1.64,1898.0,551.0,1347.0,"{""rings"": [[[-122.38758899913357, 47.669455999..."


In [7]:
%%time

# next, we need to get the locations of "our" brand businesses for analysis
biz_df = aoi_df.mdl.business.get_by_name('ace hardware')

print(biz_df.info())
biz_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   LOCNUM               35 non-null     object  
 1   CONAME               35 non-null     object  
 2   NAICS                35 non-null     object  
 3   SIC                  35 non-null     object  
 4   SOURCE               35 non-null     object  
 5   PUBPRV               35 non-null     object  
 6   FRNCOD               35 non-null     object  
 7   ISCODE               35 non-null     object  
 8   CITY                 35 non-null     object  
 9   ZIP                  35 non-null     object  
 10  STATE                35 non-null     object  
 11  SHAPE                35 non-null     geometry
 12  location_id          35 non-null     object  
 13  brand_name           35 non-null     object  
 14  brand_name_category  35 non-null     object  
dtypes: geometry(1), object(14

,LOCNUM,CONAME,NAICS,SIC,SOURCE,PUBPRV,FRNCOD,ISCODE,CITY,ZIP,STATE,SHAPE,location_id,brand_name,brand_name_category
0,174841932,ACE HARDWARE,44413005,525104,Data Axle,,,,LAKE STEVENS,98258,WA,"{""x"": -122.1068519999136, ""y"": 47.998917000209...",174841932,ACE HARDWARE,ACE HARDWARE
1,403513308,ACE HARDWARE,44413005,525104,Data Axle,,,,EVERETT,98203,WA,"{""x"": -122.21200349958885, ""y"": 47.95310699996...",403513308,ACE HARDWARE,ACE HARDWARE
2,668942261,ACE HARDWARE,44413005,525104,Data Axle,,,,MUKILTEO,98275,WA,"{""x"": -122.28130350037961, ""y"": 47.88311400023...",668942261,ACE HARDWARE,ACE HARDWARE
3,698680865,ACE HARDWARE,44413005,525104,Data Axle,,,,EVERETT,98208,WA,"{""x"": -122.20738199982871, ""y"": 47.89885499975...",698680865,ACE HARDWARE,ACE HARDWARE
4,700121193,ACE HARDWARE,44413005,525104,Data Axle,,,,NORMANDY PARK,98148,WA,"{""x"": -122.33713949958971, ""y"": 47.42394300019...",700121193,ACE HARDWARE,ACE HARDWARE


In [8]:
%%time

# since we are modeling the relationship of the customer to the retail landscape, we next also need the location of competitors as well
comp_df = aoi_df.mdl.business.get_competition(biz_df, local_threshold=3)
comp_df.spatial.set_geometry('SHAPE')

print(comp_df.info())
comp_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   LOCNUM               72 non-null     object  
 1   CONAME               72 non-null     object  
 2   NAICS                72 non-null     object  
 3   SIC                  72 non-null     object  
 4   SOURCE               72 non-null     object  
 5   PUBPRV               72 non-null     object  
 6   FRNCOD               72 non-null     object  
 7   ISCODE               72 non-null     object  
 8   CITY                 72 non-null     object  
 9   ZIP                  72 non-null     object  
 10  STATE                72 non-null     object  
 11  SHAPE                72 non-null     geometry
 12  location_id          72 non-null     object  
 13  brand_name           72 non-null     object  
 14  brand_name_category  72 non-null     object  
dtypes: geometry(1), object(14

,LOCNUM,CONAME,NAICS,SIC,SOURCE,PUBPRV,FRNCOD,ISCODE,CITY,ZIP,STATE,SHAPE,location_id,brand_name,brand_name_category
0,002890986,MC LENDON HARDWARE,44413005,525104,Data Axle,,,,SUMNER,98390,WA,"{""x"": -122.24236500031, ""y"": 47.20460400019538...",002890986,MC LENDON HARDWARE,local_brand
1,006128854,MCLENDON HARDWARE INC,44413005,525104,Data Axle,,,,RENTON,98057,WA,"{""x"": -122.21407799973574, ""y"": 47.47792500017...",006128854,MCLENDON HARDWARE INC,local_brand
2,174262691,GATEWAY TRUE VALUE HARDWARE,44413005,525104,Data Axle,,,,ENUMCLAW,98022,WA,"{""x"": -121.98761549958994, ""y"": 47.20199399976...",174262691,GATEWAY TRUE VALUE HARDWARE,local_brand
3,174471722,TWEEDY & POPP HARDWARE,44413005,525104,Data Axle,,,,SEATTLE,98103,WA,"{""x"": -122.33571339982164, ""y"": 47.66129593026...",174471722,TWEEDY & POPP HARDWARE,local_brand
4,174585992,MAPLE LEAF HARDWARE,44413005,525104,Data Axle,,,,SEATTLE,98115,WA,"{""x"": -122.31726749977874, ""y"": 47.69433000003...",174585992,MAPLE LEAF HARDWARE,local_brand


In [9]:
%%time

# now, we need to get the geographic relationshp between brand loations and the nearby competitors
biz_near_comp_prefix = 'biz_comp'
biz_near_comp_df = biz_df.mdl.proximity.get_nearest(comp_df, origin_id_column='LOCNUM', near_prefix=biz_near_comp_prefix, 
                                                    destination_id_column='LOCNUM', destination_count=6,
                                                    destination_columns_to_keep=['brand_name', 'brand_name_category'])

# in this case, side of the street is not as relevant, so we can drop these columns
biz_near_comp_df.drop(columns=[c for c in biz_near_comp_df.columns if 'side_street' in c], inplace=True)

print(biz_near_comp_df.info())
biz_near_comp_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35 entries, 0 to 34
Data columns (total 45 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   LOCNUM                            35 non-null     object  
 1   CONAME                            35 non-null     object  
 2   NAICS                             35 non-null     object  
 3   SIC                               35 non-null     object  
 4   SOURCE                            35 non-null     object  
 5   PUBPRV                            35 non-null     object  
 6   FRNCOD                            35 non-null     object  
 7   ISCODE                            35 non-null     object  
 8   CITY                              35 non-null     object  
 9   ZIP                               35 non-null     object  
 10  STATE                             35 non-null     object  
 11  location_id                       35 non-null     object  
 

,LOCNUM,CONAME,NAICS,SIC,SOURCE,PUBPRV,FRNCOD,ISCODE,CITY,ZIP,...,biz_comp_proximity_traveltime_05,biz_comp_proximity_kilometers_05,biz_comp_brand_name_05,biz_comp_brand_name_category_05,biz_comp_destination_id_06,biz_comp_proximity_traveltime_06,biz_comp_proximity_kilometers_06,biz_comp_brand_name_06,biz_comp_brand_name_category_06,SHAPE
0,174841932,ACE HARDWARE,44413005,525104,Data Axle,,,,LAKE STEVENS,98258,...,12.027158,9.369407,MANOR HARDWARE & CONSTRUCTION,local_brand,502050909,12.507132,12.695267,MCDANIELS DO IT CTR,local_brand,"{""x"": -122.1068519999136, ""y"": 47.998917000209..."
1,403513308,ACE HARDWARE,44413005,525104,Data Axle,,,,EVERETT,98203,...,10.140645,9.116069,AIRCRAFT HARDWARE,local_brand,426702925,14.948232,14.696837,SENTRY DOOR & HARDWARE LLC,local_brand,"{""x"": -122.21200349958885, ""y"": 47.95310699996..."
2,668942261,ACE HARDWARE,44413005,525104,Data Axle,,,,MUKILTEO,98275,...,15.507543,14.656761,CHOWN HARDWARE,local_brand,405053703,16.550462,16.310112,MANOR HARDWARE & CONSTRUCTION,local_brand,"{""x"": -122.28130350037961, ""y"": 47.88311400023..."
3,698680865,ACE HARDWARE,44413005,525104,Data Axle,,,,EVERETT,98208,...,12.752698,11.623863,KING DOOR & HARDWARE,local_brand,415082473,13.627706,12.553305,CLEARVIEW TRUE VALUE HARDWARE,local_brand,"{""x"": -122.20738199982871, ""y"": 47.89885499975..."
4,700121193,ACE HARDWARE,44413005,525104,Data Axle,,,,NORMANDY PARK,98148,...,19.220236,15.832812,MCLENDON HARDWARE INC,local_brand,715313723,19.790454,19.105209,EARTHWISE ARCHITECTURAL SLVG,local_brand,"{""x"": -122.33713949958971, ""y"": 47.42394300019..."


In [10]:
%%time

# now, to model the effects of cannibalization, we need the geographic relationship between the customer and all of the brand locations
bg_near_biz_keep_cols = [c for c in biz_near_comp_df.columns if c.startswith(biz_near_comp_prefix)]
bg_near_biz_df = bg_df.mdl.proximity.get_nearest(biz_near_comp_df, origin_id_column='ID', destination_count=3, 
                                                 near_prefix='brand', destination_columns_to_keep=bg_near_biz_keep_cols)

print(bg_near_biz_df.info())
bg_near_biz_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2480 entries, 0 to 2479
Columns: 128 entries, ID to SHAPE
dtypes: float64(62), geometry(1), int64(6), object(59)
memory usage: 2.4+ MB
None
CPU times: user 7.91 s, sys: 1.58 s, total: 9.49 s
Wall time: 2min 35s


,ID,NAME,TOTPOP_CY,GQPOP_CY,DIVINDX_CY,TOTHH_CY,AVGHHSZ_CY,MEDHINC_CY,AVGHINC_CY,PCI_CY,...,brand_biz_comp_proximity_traveltime_05_03,brand_biz_comp_proximity_kilometers_05_03,brand_biz_comp_brand_name_05_03,brand_biz_comp_brand_name_category_05_03,brand_biz_comp_destination_id_06_03,brand_biz_comp_proximity_traveltime_06_03,brand_biz_comp_proximity_kilometers_06_03,brand_biz_comp_brand_name_06_03,brand_biz_comp_brand_name_category_06_03,SHAPE
0,530330012001,530330012.001,1342.0,0.0,62.5,571.0,2.35,95936.0,129349.0,55036.0,...,6.031290,3.451767,STONEWAY HARDWARE & SUPPLY,local_brand,403372939,6.106129,3.894151,STONEWAY HARDWARE,local_brand,"{""rings"": [[[-122.31243049957185, 47.704870499..."
1,530330012002,530330012.002,1392.0,110.0,53.1,571.0,2.25,107276.0,149116.0,61313.0,...,6.031290,3.451767,STONEWAY HARDWARE & SUPPLY,local_brand,403372939,6.106129,3.894151,STONEWAY HARDWARE,local_brand,"{""rings"": [[[-122.3178410004258, 47.7076489995..."
2,530330032001,530330032.001,1298.0,0.0,45.6,515.0,2.52,132945.0,187017.0,74202.0,...,8.825598,6.588668,TWEEDY & POPP HARDWARE,local_brand,741123825,9.469271,5.822009,BUILDERS HARDWARE & SUPPLY CO,local_brand,"{""rings"": [[[-122.38754700019909, 47.675934999..."
3,530330032002,530330032.002,1115.0,2.0,36.7,474.0,2.35,108057.0,173682.0,73838.0,...,8.825598,6.588668,TWEEDY & POPP HARDWARE,local_brand,741123825,9.469271,5.822009,BUILDERS HARDWARE & SUPPLY CO,local_brand,"{""rings"": [[[-122.38758899913357, 47.671643999..."
4,530330032003,530330032.003,2365.0,801.0,60.9,775.0,2.02,19815.0,43614.0,15269.0,...,8.825598,6.588668,TWEEDY & POPP HARDWARE,local_brand,741123825,9.469271,5.822009,BUILDERS HARDWARE & SUPPLY CO,local_brand,"{""rings"": [[[-122.38758899913357, 47.669455999..."


In [11]:
%%time
# finally, to include the effects of competition, we need the relationship between the customers and the competitive locations
bg_near_df = bg_near_biz_df.mdl.proximity.get_nearest(comp_df, origin_id_column='ID', near_prefix='comp', destination_count=6,
                                                      destination_columns_to_keep=['brand_name', 'brand_name_category'])

print(bg_near_df.info())
bg_near_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2480 entries, 0 to 2479
Columns: 170 entries, ID to SHAPE
dtypes: float64(74), geometry(1), int64(18), object(77)
memory usage: 3.2+ MB
None
CPU times: user 11 s, sys: 2.12 s, total: 13.2 s
Wall time: 3min 29s


,ID,NAME,TOTPOP_CY,GQPOP_CY,DIVINDX_CY,TOTHH_CY,AVGHHSZ_CY,MEDHINC_CY,AVGHINC_CY,PCI_CY,...,comp_brand_name_05,comp_brand_name_category_05,comp_destination_id_06,comp_proximity_traveltime_06,comp_proximity_kilometers_06,comp_proximity_side_street_left_06,comp_proximity_side_street_right_06,comp_brand_name_06,comp_brand_name_category_06,SHAPE
0,530330012001,530330012.001,1342.0,0.0,62.5,571.0,2.35,95936.0,129349.0,55036.0,...,CITY PEOPLE SANDPOINT TRUE VL,local_brand,174471722,12.082842,8.011265,1,0,TWEEDY & POPP HARDWARE,local_brand,"{""rings"": [[[-122.31243049957185, 47.704870499..."
1,530330012002,530330012.002,1392.0,110.0,53.1,571.0,2.25,107276.0,149116.0,61313.0,...,TWEEDY & POPP HARDWARE,local_brand,301450011,11.571473,5.997022,1,0,STONEWAY HARDWARE & SUPPLY,local_brand,"{""rings"": [[[-122.3178410004258, 47.7076489995..."
2,530330032001,530330032.001,1298.0,0.0,45.6,515.0,2.52,132945.0,187017.0,74202.0,...,HOME BUILDERS CTR,local_brand,174610261,9.182731,5.809547,1,0,BUILDERS HARDWARE & SUPPLY CO,local_brand,"{""rings"": [[[-122.38754700019909, 47.675934999..."
3,530330032002,530330032.002,1115.0,2.0,36.7,474.0,2.35,108057.0,173682.0,73838.0,...,GREENWOOD TRUE VALUE HARDWARE,local_brand,174610261,8.619602,5.286317,1,0,BUILDERS HARDWARE & SUPPLY CO,local_brand,"{""rings"": [[[-122.38758899913357, 47.671643999..."
4,530330032003,530330032.003,2365.0,801.0,60.9,775.0,2.02,19815.0,43614.0,15269.0,...,BUILDERS HARDWARE & SUPPLY CO,local_brand,452112691,8.379126,4.286343,1,0,GREENWOOD TRUE VALUE HARDWARE,local_brand,"{""rings"": [[[-122.38758899913357, 47.669455999..."


This is what one *single* record looks like after this data processing.

|     | row_name                                  | row_value                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|----:|:------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|   0 | ID                                        | 530330012001                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
|   1 | NAME                                      | 530330012.001                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
|   2 | TOTPOP_CY                                 | 1342.0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
|   3 | GQPOP_CY                                  | 0.0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|   4 | DIVINDX_CY                                | 62.5                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
|   5 | TOTHH_CY                                  | 571.0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|   6 | AVGHHSZ_CY                                | 2.35                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
|   7 | MEDHINC_CY                                | 95936.0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
|   8 | AVGHINC_CY                                | 129349.0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|   9 | PCI_CY                                    | 55036.0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
|  10 | TOTHU_CY                                  | 621.0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|  11 | OWNER_CY                                  | 333.0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|  12 | RENTER_CY                                 | 238.0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|  13 | VACANT_CY                                 | 50.0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
|  14 | MEDVAL_CY                                 | 643634.0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|  15 | AVGVAL_CY                                 | 647973.0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|  16 | POPGRW10CY                                | 0.91                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
|  17 | HHGRW10CY                                 | 0.73                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
|  18 | FAMGRW10CY                                | 0.94                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
|  19 | DPOP_CY                                   | 1112.0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
|  20 | DPOPWRK_CY                                | 457.0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|  21 | DPOPRES_CY                                | 655.0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|  22 | brand_destination_id_01                   | 664387990                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|  23 | brand_proximity_traveltime_01             | 5.781369151051234                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|  24 | brand_proximity_kilometers_01             | 3.065280748467734                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|  25 | brand_proximity_side_street_left_01       | 1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|  26 | brand_proximity_side_street_right_01      | 0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|  27 | brand_biz_comp_destination_id_01_01       | 422428806                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|  28 | brand_biz_comp_proximity_traveltime_01_01 | 6.997511190266377                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|  29 | brand_biz_comp_proximity_kilometers_01_01 | 4.68810303303905                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
|  30 | brand_biz_comp_brand_name_01_01           | TOWN CENTER HARDWARE                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
|  31 | brand_biz_comp_brand_name_category_01_01  | local_brand                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  32 | brand_biz_comp_destination_id_02_01       | 174585992                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|  33 | brand_biz_comp_proximity_traveltime_02_01 | 7.904152261857584                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|  34 | brand_biz_comp_proximity_kilometers_02_01 | 4.491627725387397                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|  35 | brand_biz_comp_brand_name_02_01           | MAPLE LEAF HARDWARE                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|  36 | brand_biz_comp_brand_name_category_02_01  | local_brand                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  37 | brand_biz_comp_destination_id_03_01       | 979005980                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|  38 | brand_biz_comp_proximity_traveltime_03_01 | 10.513539781226134                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
|  39 | brand_biz_comp_proximity_kilometers_03_01 | 7.826232264740732                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|  40 | brand_biz_comp_brand_name_03_01           | CITY PEOPLE SANDPOINT TRUE VL                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
|  41 | brand_biz_comp_brand_name_category_03_01  | local_brand                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  42 | brand_biz_comp_destination_id_04_01       | 174471722                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|  43 | brand_biz_comp_proximity_traveltime_04_01 | 12.79201251481559                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|  44 | brand_biz_comp_proximity_kilometers_04_01 | 8.541096517718097                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|  45 | brand_biz_comp_brand_name_04_01           | TWEEDY & POPP HARDWARE                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
|  46 | brand_biz_comp_brand_name_category_04_01  | local_brand                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  47 | brand_biz_comp_destination_id_05_01       | 743807120                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|  48 | brand_biz_comp_proximity_traveltime_05_01 | 13.204917775457233                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
|  49 | brand_biz_comp_proximity_kilometers_05_01 | 7.752451672277373                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|  50 | brand_biz_comp_brand_name_05_01           | TWEEDY & POPP HDWR AT CROWN HL                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
|  51 | brand_biz_comp_brand_name_category_05_01  | local_brand                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  52 | brand_biz_comp_destination_id_06_01       | 741693609                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|  53 | brand_biz_comp_proximity_traveltime_06_01 | 13.209103423198641                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
|  54 | brand_biz_comp_proximity_kilometers_06_01 | 7.753149280208326                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|  55 | brand_biz_comp_brand_name_06_01           | TWEEDTY & POPP HARDWARE                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
|  56 | brand_biz_comp_brand_name_category_06_01  | local_brand                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  57 | brand_destination_id_02                   | 251373601                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|  58 | brand_proximity_traveltime_02             | 15.653280841999262                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
|  59 | brand_proximity_kilometers_02             | 12.925204735877129                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
|  60 | brand_proximity_side_street_left_02       | 0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|  61 | brand_proximity_side_street_right_02      | 1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|  62 | brand_biz_comp_destination_id_01_02       | 424970237                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|  63 | brand_biz_comp_proximity_traveltime_01_02 | 6.965391197222009                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|  64 | brand_biz_comp_proximity_kilometers_01_02 | 4.3692527910064305                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
|  65 | brand_biz_comp_brand_name_01_02           | MANOR HARDWARE & CONSTR SUPL                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
|  66 | brand_biz_comp_brand_name_category_01_02  | local_brand                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  67 | brand_biz_comp_destination_id_02_02       | 422428806                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|  68 | brand_biz_comp_proximity_traveltime_02_02 | 7.646735332880812                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|  69 | brand_biz_comp_proximity_kilometers_02_02 | 4.977588309448738                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|  70 | brand_biz_comp_brand_name_02_02           | TOWN CENTER HARDWARE                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
|  71 | brand_biz_comp_brand_name_category_02_02  | local_brand                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  72 | brand_biz_comp_destination_id_03_02       | 700299074                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|  73 | brand_biz_comp_proximity_traveltime_03_02 | 10.109385541700453                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
|  74 | brand_biz_comp_proximity_kilometers_03_02 | 6.127033454794458                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|  75 | brand_biz_comp_brand_name_03_02           | HUNTER'Z HARDWARE INC                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|  76 | brand_biz_comp_brand_name_category_03_02  | local_brand                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  77 | brand_biz_comp_destination_id_04_02       | 717859200                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|  78 | brand_biz_comp_proximity_traveltime_04_02 | 10.381830744719842                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
|  79 | brand_biz_comp_proximity_kilometers_04_02 | 6.662785225451954                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|  80 | brand_biz_comp_brand_name_04_02           | CHOWN HARDWARE                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
|  81 | brand_biz_comp_brand_name_category_04_02  | local_brand                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  82 | brand_biz_comp_destination_id_05_02       | 933642043                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|  83 | brand_biz_comp_proximity_traveltime_05_02 | 17.728547072127252                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
|  84 | brand_biz_comp_proximity_kilometers_05_02 | 15.964722334252032                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
|  85 | brand_biz_comp_brand_name_05_02           | AIRCRAFT HARDWARE                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|  86 | brand_biz_comp_brand_name_category_05_02  | local_brand                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
|  87 | brand_biz_comp_destination_id_06_02       | 804388551                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|  88 | brand_biz_comp_proximity_traveltime_06_02 | 18.299597033783428                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
|  89 | brand_biz_comp_proximity_kilometers_06_02 | 14.360034203609558                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
|  90 | brand_biz_comp_brand_name_06_02           | MCLENDON HARDWARE                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|  91 | brand_biz_comp_brand_name_category_06_02  | MCLENDON HARDWARE                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|  92 | brand_destination_id_03                   | 174611913                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|  93 | brand_proximity_traveltime_03             | 16.67170939213051                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|  94 | brand_proximity_kilometers_03             | 10.71773167377613                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|  95 | brand_proximity_side_street_left_03       | 1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|  96 | brand_proximity_side_street_right_03      | 0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
|  97 | brand_biz_comp_destination_id_01_03       | 741123825                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|  98 | brand_biz_comp_proximity_traveltime_01_03 | 1.4268028067157457                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
|  99 | brand_biz_comp_proximity_kilometers_01_03 | 0.7531675169520554                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| 100 | brand_biz_comp_brand_name_01_03           | BUILDERS HARDWARE & SUPPLY CO                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| 101 | brand_biz_comp_brand_name_category_01_03  | local_brand                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 102 | brand_biz_comp_destination_id_02_03       | 174612515                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| 103 | brand_biz_comp_proximity_traveltime_02_03 | 3.640653465153758                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 104 | brand_biz_comp_proximity_kilometers_02_03 | 2.3492601984257684                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| 105 | brand_biz_comp_brand_name_02_03           | HOME BUILDERS CTR                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 106 | brand_biz_comp_brand_name_category_02_03  | local_brand                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 107 | brand_biz_comp_destination_id_03_03       | 174610261                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| 108 | brand_biz_comp_proximity_traveltime_03_03 | 4.883588321957539                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 109 | brand_biz_comp_proximity_kilometers_03_03 | 2.614963831210113                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 110 | brand_biz_comp_brand_name_03_03           | BUILDERS HARDWARE & SUPPLY CO                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| 111 | brand_biz_comp_brand_name_category_03_03  | local_brand                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 112 | brand_biz_comp_destination_id_04_03       | 710798192                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| 113 | brand_biz_comp_proximity_traveltime_04_03 | 5.9380327916542255                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| 114 | brand_biz_comp_proximity_kilometers_04_03 | 3.3472345861454236                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| 115 | brand_biz_comp_brand_name_04_03           | CONTRACT HARDWARE INC                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| 116 | brand_biz_comp_brand_name_category_04_03  | local_brand                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 117 | brand_biz_comp_destination_id_05_03       | 301450011                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| 118 | brand_biz_comp_proximity_traveltime_05_03 | 6.031290240152461                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 119 | brand_biz_comp_proximity_kilometers_05_03 | 3.4517668631407106                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| 120 | brand_biz_comp_brand_name_05_03           | STONEWAY HARDWARE & SUPPLY                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 121 | brand_biz_comp_brand_name_category_05_03  | local_brand                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 122 | brand_biz_comp_destination_id_06_03       | 403372939                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| 123 | brand_biz_comp_proximity_traveltime_06_03 | 6.106128931877896                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 124 | brand_biz_comp_proximity_kilometers_06_03 | 3.8941508004200953                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| 125 | brand_biz_comp_brand_name_06_03           | STONEWAY HARDWARE                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 126 | brand_biz_comp_brand_name_category_06_03  | local_brand                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 127 | comp_destination_id_01                    | 174585992                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| 128 | comp_proximity_traveltime_01              | 4.003591696083123                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 129 | comp_proximity_kilometers_01              | 1.8210412021187152                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| 130 | comp_proximity_side_street_left_01        | 0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 131 | comp_proximity_side_street_right_01       | 1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 132 | comp_brand_name_01                        | MAPLE LEAF HARDWARE                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| 133 | comp_brand_name_category_01               | local_brand                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 134 | comp_destination_id_02                    | 743807120                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| 135 | comp_proximity_traveltime_02              | 9.495089102386622                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 136 | comp_proximity_kilometers_02              | 5.2534760287088895                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| 137 | comp_proximity_side_street_left_02        | 0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 138 | comp_proximity_side_street_right_02       | 1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 139 | comp_brand_name_02                        | TWEEDY & POPP HDWR AT CROWN HL                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| 140 | comp_brand_name_category_02               | local_brand                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 141 | comp_destination_id_03                    | 741693609                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| 142 | comp_proximity_traveltime_03              | 9.49927475012803                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| 143 | comp_proximity_kilometers_03              | 5.254173636639843                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 144 | comp_proximity_side_street_left_03        | 0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 145 | comp_proximity_side_street_right_03       | 1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 146 | comp_brand_name_03                        | TWEEDTY & POPP HARDWARE                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| 147 | comp_brand_name_category_03               | local_brand                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 148 | comp_destination_id_04                    | 452112691                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| 149 | comp_proximity_traveltime_04              | 11.221143321512592                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| 150 | comp_proximity_kilometers_04              | 6.06023156683205                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| 151 | comp_proximity_side_street_left_04        | 1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 152 | comp_proximity_side_street_right_04       | 0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 153 | comp_brand_name_04                        | GREENWOOD TRUE VALUE HARDWARE                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| 154 | comp_brand_name_category_04               | local_brand                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 155 | comp_destination_id_05                    | 979005980                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| 156 | comp_proximity_traveltime_05              | 11.817862247463905                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| 157 | comp_proximity_kilometers_05              | 8.681335163413316                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 158 | comp_proximity_side_street_left_05        | 0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 159 | comp_proximity_side_street_right_05       | 1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 160 | comp_brand_name_05                        | CITY PEOPLE SANDPOINT TRUE VL                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| 161 | comp_brand_name_category_05               | local_brand                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 162 | comp_destination_id_06                    | 174471722                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| 163 | comp_proximity_traveltime_06              | 12.082841956853715                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| 164 | comp_proximity_kilometers_06              | 8.011264874686852                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 165 | comp_proximity_side_street_left_06        | 1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 166 | comp_proximity_side_street_right_06       | 0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 167 | comp_brand_name_06                        | TWEEDY & POPP HARDWARE                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| 168 | comp_brand_name_category_06               | local_brand                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 169 | SHAPE                                     | {"type": "MultiPolygon", "coordinates": [[[[-122.31243049957185, 47.704870499507166], [-122.31225500020636, 47.701248000111725], [-122.31266899958482, 47.70124999940139], [-122.31497300042217, 47.701266999708245], [-122.31721199916554, 47.70128200011668], [-122.31769200046288, 47.70128599929785], [-122.3178009993445, 47.70491799971879], [-122.3178410004258, 47.70764899958031], [-122.31785299922304, 47.70855900044265], [-122.31787299931453, 47.709787999477044], [-122.31795300057878, 47.71017799934367], [-122.3178909997562, 47.71094699959987], [-122.31790999912445, 47.71219799940442], [-122.31519400035508, 47.71219599932575], [-122.31380399938674, 47.71217299992706], [-122.31243499923309, 47.71216500021267], [-122.31243400030648, 47.71133100012933], [-122.31243299958327, 47.710354999259536], [-122.31243200065667, 47.708526999963134], [-122.31243099993345, 47.706698000180616], [-122.31243099993345, 47.70552900001329], [-122.31243049957185, 47.704870499507166]]]]} |

In [12]:
# from here, we need to add a performance metric (customer count or sales by store by block group), and then we can start applying a modeling workflow
# A typical workflow includes initial feature selection (SelectKBest), dimensionality reduction (PCA), clustering (KMeans), forecasting by cluster (XGBoost), and final feature selection (RFECV).
save_df = bg_near_df.copy()
save_df[save_df.spatial.name] = save_df[save_df.spatial.name].apply(lambda geom: geom.JSON)
save_df.to_parquet(dir_int/'block_group_enriched_near.parquet')

In [ ]:
rdf = save_df.iloc[0].to_frame().reset_index()
rdf.columns = ['row_name', 'row_value']
print(rdf.to_markdown())